# Visualization of DNA movement in dividing Sufolobus

In this notebook, we load the h5 file containing the results and generate individual visualization.

In [ ]:
%load_ext autoreload
%autoreload 2

Setup the path to the destination folder

In [ ]:
from pathlib import Path
import dnasufo 
import matplotlib.pyplot as plt
from ipyfilechooser import FileChooser

dst = Path("")
fc = FileChooser(dst, select_desc="Destination")
display(fc)


In [ ]:
import pandas as pd
dst = Path(fc.selected)
filelist = pd.read_csv(dst / 'filelist.csv')

List all the files in the destination folder

In [ ]:
items = dnasufo.inspect_result(dst)
print(f"There are {len(items)} items in '{dst}'.")

Make a figure for a selected a sequences.

In [ ]:
plt.rc('font', size=6)
mq.figure(dst, 17, frame="auto")

Save all figures in a multipage pdf

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

plt.rc('font', size=6)
with PdfPages(dst / 'figures.pdf') as pdf:
    for index in items:
        plt.clf()
        dnasufo.figure(dst, index, frame="auto")
        pdf.savefig()
        plt.close()

Inspect a single strip

In [ ]:
dnasufo.strip(dst, 17, "Greys", selection="auto", quiver=False)


Save all strips as individual pdf files

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

plt.rc('font', size=6)
with PdfPages(dst / 'strips.pdf') as pdf:
    for index in items:
        plt.clf()
        mq.strip(dst, index, "Greys", selection="auto", quiver=False)
        pdf.savefig()
        plt.close()

Visualize the image and mask using napari

In [ ]:
# import napari
# import numpy as np
# img, cell_mask, cell_trj, diff, flow, rho, div, blob_labels, blob_trj = mq.load_result(
#     filename, name
# )
# vector = mq.make_vector(rho,4)
# v = napari.view_image(img, channel_axis=1)
# v.add_labels(cell_mask.squeeze().astype(np.uint64))
# v.add_vectors(vector, edge_width=0.1, edge_color="white", length=1)

### Export all csv to excel

In [ ]:
import pandas as pd

# load and concatenate all the csv files
df = pd.concat([pd.read_csv(dst / f'{idx:06d}.csv') for idx in items if (dst / f'{idx:06d}.csv').exists()])


In [ ]:
df

Add a synchronized time

In [ ]:
ndf = []
for k, ng in enumerate(df.groupby('filename')):
    name, group = ng
    t0 = mq.split_frame(group)    
    group['sync frame'] = group['frame'] - t0
    ndf.append(group)
df = pd.concat(ndf)

Add the condition

In [ ]:
filelist = pd.read_csv('filelist.csv',usecols=['name','condition'])
filelist['filename'] = filelist['name'].apply(lambda x:Path(x).stem)
dfc = df.merge(filelist.drop('name',axis=1),on='filename',how='left')
dfc.to_csv(dst/'results_with_condition.csv')

Save all results to a excel file

In [ ]:
with pd.ExcelWriter(dst/'sheets.xlsx') as writer:
    for tab in df.columns[6:]:
        tmp = df.pivot_table(values=tab, index='frame', columns='filename')
        tmp.to_excel(writer,sheet_name=tab)
        # smoothed version
        for c in tmp.columns:
            tmp[c] = mq.ndi.gaussian_filter1d(tmp[c], 5)
        tmp.to_excel(writer,sheet_name=f"{tab}-smoothed")

In [ ]:
pd.set_option('future.no_silent_downcasting', True)
with pd.ExcelWriter(dst/'sheet.xlsx') as writer:
    for n,g in df.groupby('filename'):
        for c in df.columns[7:-1]:
            g[f'{c} smoothed'] = mq.ndi.gaussian_filter1d(g[c].replace(pd.NA, 0.0).infer_objects(copy=False), 5)
        g.to_excel(writer, sheet_name=Path(n).stem)

Display each measurement by condition

In [ ]:
import seaborn as sns
import numpy as np
cols = dfc.columns[5:-2]
fig,ax = plt.subplots(4, int(np.ceil(len(cols)/4)))
ax = ax.ravel()
for k,c in enumerate(cols):    
    p = sns.lineplot(data=dfc[dfc["sync frame"]<100], x='sync frame', y=c, hue='condition', ax=ax[k])
    p.get_legend().remove()
fig.set_tight_layout(True)

Use seaborn to create a figure

In [ ]:
dflong = pd.melt(df.iloc[0:1200], id_vars=('filename','frame') , value_vars=df.columns[6:])
dflong.head()

In [ ]:
grid = sns.FacetGrid(data=dflong, row='filename', col='variable', sharey = False)
grid.map_dataframe(sns.lineplot, x="frame",y="value")
# grid.set_titles(template="", row_template='{row_var}{row_name}',col_template='{col_var}{col_name}')
grid.set_titles("", col_template='{col_name}')

In [ ]:
facet_kws={'sharey': False, 'sharex': True}
sns.relplot(data=dflong, x="frame", y="value", col="filename", row="variable", kind='line', facet_kws=facet_kws)